In [9]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F
from transformers import  DistilBertModel,DistilBertTokenizerFast
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import pandas as pd
import random

In [ ]:
class jointBert(pl.LightningModule):
    
    def __init
    
    

In [ ]:
class NLU_Dataset(pl.LightningDataModule):
    
    def __init__(self, data_dir: str = "path/to/dir", batch_size: int = 32):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

    def setup(self, stage: Optional[str] = None):
        self.train = nluDataset('../data/multiATIS/splits/train_EN.tsv',
                                'distilbert-base-multilingual-cased',56,2)
        
        self.train = nluDataset('../data/multiATIS/splits/dev_EN.tsv',
                                'distilbert-base-multilingual-cased',56,2)

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size)

In [2]:
class jointBert(nn.Module):

    def __init__(self, model_name):

        super(jointBert,self).__init__()
        
        self.encoder = DistilBertModel.from_pretrained(model_name,return_dict=True,output_hidden_states=True)
        self.intent_dropout = nn.Dropout(0.2)
        self.intent_FC = nn.Linear(768, 17)
 

        # slots layer
        self.slots_dropout = nn.Dropout(0.2)
        self.slots_FC = nn.Linear(768, 159)
        
        self.intent_loss_fn = nn.CrossEntropyLoss()
        self.slot_loss_fn = nn.CrossEntropyLoss()
        
        self.jlc = 0.5

    
    def forward(self, input_ids, attention_mask , intent_target, slots_target):

        encoded_output = self.encoder(input_ids, attention_mask)

        #intent data flow
        intent_hidden = encoded_output[0][:,0]
        intent_logits = self.intent_FC(self.intent_dropout(F.relu(intent_hidden)))
        
        
        # accumulating intent classification loss 
        intent_loss = self.intent_loss_fn(intent_logits, intent_target)
        intent_pred = torch.argmax(nn.Softmax(dim=1)(intent_logits), axis=1)
        

        # slots data flow 
        slots_hidden = encoded_output[0]
        slots_logits = self.slots_FC(self.slots_dropout(F.relu(slots_hidden)))
       

        # accumulating slot prediction loss
        slot_loss = self.slot_loss_fn(slots_logits.view(-1, 159), slots_target.view(-1))
        
        joint_loss = ((1-self.jlc)*intent_loss + (self.jlc)*slot_loss)
        

        return joint_loss,intent_pred,intent_loss,slot_loss


In [4]:
model = jointBert('distilbert-base-multilingual-cased').to('cuda')